K-neighbors es lo recomendado para medir similaridad en datasets grandes por su eficiencia. Además, si se quiere extender la funcionalidad hacia user-item recommendation, es ideal también. En este modelo se puede definir distintas medidas de similaridad como distancia euclidiana o cosine similarity. 
Lo más importante es escoger correctamente los features a ingresar en la matriz, deben ser relevantes. -- Falta revisar si el método tfidf tiene un tope de palabras y si se puede asignar stopwords en español. 


-- Columnas a usar: genres, developer, tags, specs

In [71]:
import pandas as pd

games = pd.read_csv('datasets/steam_games.csv')

In [72]:
games['release_date'] = pd.to_datetime(games['release_date'], errors='coerce')
games['release_year'] = games['release_date'].dt.year
games['release_year'] = games['release_year'].fillna(0).astype(int)
games['release_year'] = games['release_year'].astype('str').replace('0', '')

In [73]:
games.fillna('', inplace=True)

games.drop(games[games['title'] == ""].index, inplace=True)
games.shape

(30085, 14)

In [74]:
# Aplica la función de unión con guión a la columna 'developer'
games['developer'] = games['developer'].str.replace(r'\s+', '-', regex=True)
games['developer'] = games['developer'].str.replace(r'---', "-")

In [75]:
import re
import ast


def tolist(lst):
    text= ""
    if isinstance(lst, str) and len(lst) > 0 and '[' in lst:
        lst = lst.lower()
        lst = lst.replace("0's",'0s').replace(';','').replace('&','').replace("'em","em")
        try:
            lst =  ast.literal_eval(lst)
        except (SyntaxError, ValueError):
            return lst.lower()
        
    if isinstance(lst,list):
        for item in lst:
            while "  " in item:
                item = item.replace("  "," ")
            item = item.replace(" ", "-")
            text = text + " " + item 
            text = text.replace('"','')
        return text.strip().lower()

    return lst

In [76]:
games['specs'] = games['specs'].apply(lambda x : tolist(x))
games['genres'] = games['genres'].apply(lambda x : tolist(x))
games['tags'] = games['tags'].apply(lambda x : tolist(x))

In [77]:
# Combine relevant columns into a single 'profile'
games['profile'] = games['genres'] + " " + games['tags'] + " " + games['specs'] + " " + games['developer'] + " " + games['release_year']


games['profile_simple'] = games['genres']+ " " + + games['developer'] + " " + games['release_year']
# games['profile_simple'] = games['profile_simple'].fillna(" ")

In [78]:
games['profile_simple'].loc[1]

'free-to-play indie rpg strategy Secret-Level-SRL 2018'

In [79]:
games['profile_simple'].loc[5]

'action adventure simulation Trickjump-Games-Ltd 2018'

In [80]:
games[['id','title','profile','profile_simple']].to_csv(r'./datasets/game_profile.csv',index=False)